In [2]:
class SparseMatrix:
    def __init__(self):
        self.data: list[int] = []
        self.indices: list[int] = []
        self.indptr: list[int] = []
        self.nCols: int = 0
        self.nRows: int = 0
        self.OriginalPos = []
    
    def printFull(self):
        matrix = self.toMatrix()
        for line in matrix:
            print(line)


    def __repr__(self):
        s=""
        s+= f"row={self.nRows}, col={self.nCols}, NNZ={len(self.data)}\n"
        s+= f"data:    {self.data}\n"
        s+= f"indices: {self.indices}\n"
        s+= f"indprt:  {self.indptr}\n"
        s+= f"\n"
        s+= f"origin:  {self.originalPos}\n"
        return s

    def fromMatrix(self, m: list[list[int]]):
        self.nCols = len(m[0])
        self.nRows = len(m)

        self.indptr.append(0)

        for rowIdx in range(self.nRows):
            for colIdx in range(self.nCols):
                data = m[rowIdx][colIdx]
                if data != 0:
                    self.data.append(data)
                    self.indices.append(colIdx)
            self.indptr.append(len(self.data))
        
        self.originalPos = [i for i in range(len(self.data))]

        return self

    def toMatrix(self) -> list[list[int]]:
        matrix = [[0]*self.nCols for _ in range(self.nRows)]

        for row in range(self.nRows):
            for data, col in self.getRow(row):
                matrix[row][col] = data

        return matrix

    def getRow(self, i: int):
        #returns iterator over row i of (data, col)
        start = self.indptr[i]
        end = self.indptr[i+1]
        for j in range(start, end):
            yield (self.data[j], self.indices[j])

    def mul(self, witness: list[int]):
        assert len(witness) == self.nCols, f"lenW != nRows ({len(witness)}, {self.nCols})"
        res = []
        for i in range(self.nRows):
            s = sum([data*witness[col] for data, col in self.getRow(i)])
            res.append(s)
        return res
    

    

In [3]:
matrix = [[1, 2, 0, 0, 0, 0],
          [0, 3, 0, 4, 0, 0],
          [0, 0, 5, 6, 7, 0],
          [0, 0, 0, 0, 0, 8]]

w =       [1, 1, 1, 1, 1, 1]
w2=       [2, 2, 2, 2, 2, 2]
w3=       [2, 10, 0, 0, 0, 0]
sm = SparseMatrix().fromMatrix(matrix)

print(sm)
sm.printFull()

print("\nMult")

print(sm.mul(w))
print(sm.mul(w2))
print(sm.mul(w3))

row=4, col=6, NNZ=8
data:    [1, 2, 3, 4, 5, 6, 7, 8]
indices: [0, 1, 1, 3, 2, 3, 4, 5]
indprt:  [0, 2, 4, 7, 8]

origin:  [0, 1, 2, 3, 4, 5, 6, 7]

[1, 2, 0, 0, 0, 0]
[0, 3, 0, 4, 0, 0]
[0, 0, 5, 6, 7, 0]
[0, 0, 0, 0, 0, 8]

Mult
[3, 7, 18, 8]
[6, 14, 36, 16]
[22, 30, 0, 0]


In [4]:
from collections import Counter

def sort_by_frequency(nums):
    # Count frequencies using
    frequency = Counter(nums)
    # Create a list of indices
    indices = list(range(len(nums)))
    # Sort indices based on the frequency of the value at each index
    indices.sort(key=lambda x: (-frequency[nums[x]], nums[x]))
    
    return indices

class SparseMatrixCudaAlg:
    def __init__(self, debug = True):
        self.data: list[int] = []
        self.indices: list[int] = []
        self.indptr: list[int] = []
        self.nCols: int = 0
        self.nRows: int = 0
        self.debug = debug
    
    def __repr__(self):
        s=""
        s+= f"row={self.nRows}, col={self.nCols}, NNZ={len(self.data)}\n"
        s+= f"origin:  {self.originalPos}\n"
        s+= f"data:    {self.data}\n"
        s+= f"indices: {self.indices}\n"
        s+= f"indprt:  {self.indptr}\n"
        s+= f"\n"
        return s

    def fromMatrix(self, m: list[list[int]]):
        self.nCols = len(m[0])
        self.nRows = len(m)

        self.indptr.append(0)

        for rowIdx in range(self.nRows):
            for colIdx in range(self.nCols):
                data = m[rowIdx][colIdx]
                if data != 0:
                    self.data.append(data)
                    self.indices.append(colIdx)
            self.indptr.append(len(self.data))
        
        self.originalPos = [i for i in range(len(self.data))]
        self.preprocess()

        return self

    def preprocess(self):
        ss = lambda x: x[0] #stabilize sorting
        # sorting information
        self.colWeight=[0]*self.nCols #number of set elements in colum i
        for i in self.indices:
            self.colWeight[i] += 1
        # if self.debug:  print(f">>> {self.colWeight=}")

        self.rowWeight=[0]*self.nRows #number of set elements in row i
        for i in range(self.nRows):
            self.rowWeight[i] = self.indptr[i+1]-self.indptr[i]
        if self.debug: print(f">>> {self.rowWeight=}")

        self.rowDestination = sort_by_frequency(self.rowWeight) #contains new row order

        if self.debug: print(f">>> {self.rowDestination=}")


        #posVector: array with one element for each data, tells where it is supposed to be written after the multiplication
        self.posVector = []
        for row in range(self.nRows):
            source = self.rowDestination[row]
            start = self.indptr[source]
            end = self.indptr[source+1]
            for i in range(start, end):
                self.posVector.append(i) 

        if self.debug: print(f">>> {self.posVector=}")

        #Now, we have 3 arrays: data, col, and posVector. PosVector points the index a given element will go on an array where they are sorted by row density, eg data[i] should be written to posVector[i]
        #generate a new indptr to tell the borders of this new layout
    
        run = 0
        self.s_indptr = [0]
        for i in range(self.nRows):
            sourceRow = self.rowDestination[i]
            rowWeight = self.indptr[sourceRow+1] - self.indptr[sourceRow]
            run += rowWeight
            self.s_indptr.append(run)
    
        if self.debug: print(f">>> {self.s_indptr=}")

        #finally, we order things using the colWeight
        if self.debug: print(f">>> {self.colWeight=}")

        cols_sort_key = sort_by_frequency(self.indices)
        if self.debug: print(f">>> {cols_sort_key=}")

        nelements = len(self.data)

        self.s_cols = [0]*nelements
        self.s_data = [0]*nelements
        self.s_originalPos = [0]*nelements

        for i in range(nelements):
            self.s_cols[i] =  self.indices[cols_sort_key[i]]
            self.s_data[i] =  self.data[cols_sort_key[i]]
            self.s_originalPos[i] =  self.originalPos[cols_sort_key[i]]

        if self.debug: print(f">>> {self.s_data=}")
        if self.debug: print(f">>> {self.s_cols=}")
        if self.debug: print(f">>> {self.s_originalPos=}")

        #ready for multiplication

    def mul(self, witness):
        ss = lambda x: x[0] #stabilize sorting
        product_tmp = [self.s_data[i] * witness[self.s_cols[i]] for i in range(len(self.data))]
        #write to the correct location using posVector
        if self.debug: print(f">>> {product_tmp=}")

        product_tmp_sorted_tmp=[0]*len(self.data)
        indices_sorted_tmp = [0]*len(self.data)
        
        #move to their original position:
        for i in range(len(self.data)):
            product_tmp_sorted_tmp[self.s_originalPos[i]] = product_tmp[i]
            indices_sorted_tmp[self.s_originalPos[i]] = self.s_originalPos[i]

        if self.debug: print(f">>> {product_tmp_sorted_tmp=}")
        if self.debug: print(f">>> {indices_sorted_tmp=}")
        #move to the optimal sum position

        product_tmp_sorted=[0]*len(self.data)
        indices_sorted = [0]*len(self.data)
        for i in range(len(self.data)):
            product_tmp_sorted[i] = product_tmp_sorted_tmp[self.posVector[i]]
            indices_sorted[i] = indices_sorted_tmp[self.posVector[i]]

        if self.debug: print(f">>> {product_tmp_sorted=}")
        if self.debug: print(f">>> {indices_sorted=}")


        #sum using the new indptr
        def getRow(i: int): #CAUTION: CLOSURE!
            #returns iterator over row i of (data, col)
            start = self.s_indptr[i]
            end = self.s_indptr[i+1]
            if start==end: 
                yield 0
            else:
                for j in range(start, end):
                    yield (product_tmp_sorted[j])

        s = [0]*self.nRows
            
        for i in range(self.nRows):
            s[i] = [j for j in getRow(i)]
        if self.debug: print(f">>> {s=}")
        s = [sum(i) for i in s] #reducing
        #sort result
        sorted_sum = [a for b, a in sorted(zip(self.rowDestination, s), key=ss)]
        if self.debug: print(f">>> {sorted_sum=}")

        return sorted_sum


matrix = [[11, 22, 0,  0, 99, 0,  0],
          [0,  33, 0,  0, 44, 0,  0],
          [0,  0,  55, 0, 66, 77, 0],
          [0,  0,  0,  0, 0 , 0,  0],
          [0,  0,  0,  0, 0 , 0,  88]]

# matrix = [[1,  2,  0,  0, 9,  0,  0],
#           [0,  3,  0,  0, 4,  0,  0],
#           [0,  0,  5,  0, 6,  7,  0],
#           [0,  0,  0,  0, 0 , 0,  0],
#           [0,  0,  0,  0, 0 , 0,  8]]


smc = SparseMatrixCudaAlg().fromMatrix(matrix)
w = [1, 1, 1, 1, 1, 1, 1]
print(smc.mul(w))

print(SparseMatrix().fromMatrix(matrix).mul(w))

>>> self.rowWeight=[3, 2, 3, 0, 1]
>>> self.rowDestination=[0, 2, 3, 4, 1]
>>> self.posVector=[0, 1, 2, 5, 6, 7, 8, 3, 4]
>>> self.s_indptr=[0, 3, 6, 6, 7, 9]
>>> self.colWeight=[1, 2, 1, 0, 3, 1, 1]
>>> cols_sort_key=[2, 4, 6, 1, 3, 0, 5, 7, 8]
>>> self.s_data=[99, 44, 66, 22, 33, 11, 55, 77, 88]
>>> self.s_cols=[4, 4, 4, 1, 1, 0, 2, 5, 6]
>>> self.s_originalPos=[2, 4, 6, 1, 3, 0, 5, 7, 8]
>>> product_tmp=[99, 44, 66, 22, 33, 11, 55, 77, 88]
>>> product_tmp_sorted_tmp=[11, 22, 99, 33, 44, 55, 66, 77, 88]
>>> indices_sorted_tmp=[0, 1, 2, 3, 4, 5, 6, 7, 8]
>>> product_tmp_sorted=[11, 22, 99, 55, 66, 77, 88, 33, 44]
>>> indices_sorted=[0, 1, 2, 5, 6, 7, 8, 3, 4]
>>> s=[[11, 22, 99], [55, 66, 77], [0], [88], [33, 44]]
>>> sorted_sum=[132, 77, 198, 0, 88]
[132, 77, 198, 0, 88]
[132, 77, 198, 0, 88]


In [5]:
from random import randint, randrange

def genMatrix(nRows, nCols):
    matrix = [[0]*nCols for _ in range(nRows)]
    return matrix

def sparseRandom(matrix, NNZ):
    nRows = len(matrix)
    nCols = len(matrix[0])
    for _ in range(NNZ):
        matrix[randrange(0, nRows)][randrange(0, nCols)] = randint(0, 255)
    return matrix

def randWitness(matrix, maxdensity):
    #max density because I am lazy to check that two randoms are not the same pos. For sparse it should work.
    nCols = len(matrix[0])
    assert maxdensity <= nCols
    w = [0]*nCols
    for _ in range(maxdensity):
        w[randrange(0, maxdensity)] = randint(0, 255)
    
    return w


matrix = sparseRandom(genMatrix(200,100), 100)
w = randWitness(matrix, 50)

m = SparseMatrix().fromMatrix(matrix).mul(w)
m2 = SparseMatrixCudaAlg(debug=False).fromMatrix(matrix).mul(w)

assert m==m2

# print(m)
# print(m2)

# Inputs for C/CUDA test

In [19]:
def printC(s, array):
    print(f"{s}[{len(array)}] = {{", end='')
    print(*array, sep=", ", end="")
    print("};")

def printFull(matrix):
    for line in matrix:
        print(line)

import random
random.seed(0)

nCols = 20
nRows = 10


A_ = sparseRandom(genMatrix(nRows, nCols), 30)
# W = randWitness(A_, nCols)
W = [1]*nCols
A = SparseMatrix().fromMatrix(A_)


printFull(A_)

printC("fr_t data", A.data)
printC("size_t indices", A.indices)
printC("size_t indptr", A.indptr)
print("")
printC("fr_t witness", W)
print("")


ref = SparseMatrixCudaAlg(debug=False).fromMatrix(A_)
ref2 = SparseMatrix().fromMatrix(A_)
m = ref.mul(W)
m2 = ref2.mul(W)

assert m==m2

print(m)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 133, 0, 0]
[0, 0, 229, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 19, 0, 0]
[0, 96, 0, 0, 45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 113, 0, 0, 0, 0, 0, 170, 0, 0, 0, 0, 0, 0, 0, 32, 0]
[0, 0, 0, 75, 71, 123, 0, 0, 0, 0, 0, 0, 0, 0, 0, 207, 0, 55, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 51, 0, 37, 0, 0, 197, 0]
[0, 197, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 133, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 244, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 170, 0, 200, 0, 0, 0, 0, 0, 0, 163, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 161, 0, 48, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
fr_t data[25] = {133, 229, 7, 19, 96, 45, 113, 170, 32, 75, 71, 123, 207, 55, 51, 37, 197, 197, 133, 244, 170, 200, 163, 161, 48};
size_t indices[25] = {17, 2, 12, 17, 1, 4, 4, 10, 18, 3, 4, 5, 15, 17, 13, 15, 18, 1, 2, 16, 6, 8, 15, 6, 8};
size_t indptr[11] = {0, 1, 4, 6, 9, 14, 17, 18, 20, 23, 25};

fr_t witness[20] = {1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

# Experiments